# GPU 사용가능한지 확인

In [1]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20  Python-3.10.13 torch-2.1.1+cpu CPU
Setup complete  (12 CPUs, 31.9 GB RAM, 575.8/931.4 GB disk)


# YOlov8n모델 예측 후 객체의 정보 얻기

In [4]:
from ultralytics import YOLO
import torch

# 모델 불러오기 없으면 다운받기
model = YOLO('yolov8n.pt')

# 예측할 이미지 or 동영상
path = './sample.jpg'

# 예측 결과
results = model.predict(path, save=True)

# 객체의 데이터
for result in results:
    # 객체의 바운딩 박스 결과
    boxes = result.boxes

# 클래스 ID
object_type = []
for _ in boxes.cls:
    object_type.append(int(_))

# 객체의 좌표
yolo_point = []
for _ in boxes.xywh:
    center_x = int(_[0])
    center_y = int(_[1])
    width = int(_[2])
    hight = int(_[3])

    yolo_point.append((center_x, center_y, width, hight))


image 1/1 c:\study\blackbox\sample.jpg: 640x640 3 persons, 1 car, 157.5ms
Speed: 5.0ms preprocess, 157.5ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3


In [5]:
result_path = './' + result.save_dir.replace('\\', '/') + '/'
result_path

'./runs/detect/predict3/'

In [6]:
# Opencv 로 시각화 할때 필요한 예측 결과 이미지 or 예측 동영상
import os

img_file = os.listdir(result_path)
show_result = result_path + img_file[0]

In [7]:
# coco 데이터 구조 파악
result.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [8]:
object_type

[0, 0, 0, 2]

In [9]:
len(object_type), len(yolo_point)

(4, 4)

In [10]:
object_type

[0, 0, 0, 2]

In [11]:
yolo_point

[(295, 372, 23, 99),
 (411, 384, 34, 170),
 (440, 366, 38, 164),
 (528, 329, 39, 43)]

### 이미지내 불필요한 클래스 제거

In [12]:
# 불필요한 클래스id 인덱스 받기
drop_index = []

# 0 : 사람, 1 : 자전거, 2 : 자동차
# 3 : 오토바이, 5 : 버스, 7 : 트럭
for _ in range(0, len(object_type)):
    if object_type[_] == 0 or object_type[_] == 1 or object_type[_] == 2 or object_type[_] == 3 or object_type[_] == 5 or object_type[_] == 7:
        pass
    else:
        drop_index.append(_)

# 불필요한 클래스 인덱스 삭제
# 인덱스 번호를 안 뒤집을 경우 앞에서부터 삭제하므로
# 뒤로갈수록 해당 인덱스 존재 안해서.
drop_index.reverse()
for _ in drop_index:
    object_type.pop(_)
    yolo_point.pop(_)

### 클래스 id 정수형 -> 한글로 변환

In [13]:
# 클래스id -> 한글로 변환
for _ in range(0, len(object_type)):
    if object_type[_] == 0:
        object_type[_] = '사람'

    elif object_type[_] == 1:
        object_type[_] = '자전거'

    elif object_type[_] == 2:
        object_type[_] = '자동차'

    elif object_type[_] == 3:
        object_type[_] = '오토바이'

    elif object_type[_] == 5:
        object_type[_] = '버스'

    elif object_type[_] == 7:
        object_type[_] = '트럭'       

object_type


['사람', '사람', '사람', '자동차']

### Yolo식 좌표는 center_x, center_y, w, h -> 바운딩박스 4개의 점 좌표로 변환

In [14]:
# 사각형의 4개의 점 좌표 구하기.
# x1 : 왼쪽 상단, x2 : 오른쪽 상단
# x3 : 왼쪽 하단, x4 : 오른쪽 하단

# 하나의 클래스 기준 4개의 점 좌표 저장.
point = []

for raw in yolo_point:
    x = raw[0]
    y = raw[1]
    w = raw[2]
    h = raw[3]
    x1 = (int(x - (w/2)), int(y + (h/2)))
    x2 = (int(x + (w/2)), int(y + (h/2)))
    x3 = (int(x - (w/2)), int(y - (h/2)))
    x4 = (int(x + (w/2)), int(y - (h/2)))

    point.append((x1, x2, x3, x4))

In [15]:
point

[((283, 421), (306, 421), (283, 322), (306, 322)),
 ((394, 469), (428, 469), (394, 299), (428, 299)),
 ((421, 448), (459, 448), (421, 284), (459, 284)),
 ((508, 350), (547, 350), (508, 307), (547, 307))]

In [16]:
point[0]

((283, 421), (306, 421), (283, 322), (306, 322))

### 알고리즘 구현

In [17]:
# opencv식 워닝존 좌표
warning_point = [(270, 310), (12, 445), (610, 445), (383, 310)]

# 변환 워닝존 좌표
warning_point = [(12, 310), (270, 445), (383, 445), (610, 310)]

In [18]:
# !pip install shapely

In [19]:
from shapely.geometry import Polygon, Point

def warning_zone(point, polygon):
    """
    point: [x, y] 좌표 값을 가진 리스트
    polygon: 다각형의 꼭지점 좌표를 가진 리스트
    """

    # 점의 좌표를 사용하여 점의 객체를 만듭니다.
    point_obj = Point(point)

    # 다각형의 꼭지점 좌표를 사용하여 다각형의 경계를 구합니다.
    polygon_path = Polygon(polygon)

    # 점의 객체가 다각형의 경계 안에 있는지 확인합니다.
    result = polygon_path.contains(point_obj)
    print(result)
    # print('워닝존 : ', polygon)
    return result

# 워닝존
rectangle = [(280, 290), (20, 470), (620, 470), (380, 290)]

# 데이터 클래스 하나 4개의 점
# square = point

result_case = []
# 좌표가 사다리꼴 안에 있는지 확인합니다.
for square, result_type in zip(point, object_type):
    for p in square:
        point_obj = Point(p)
        if warning_zone(point_obj, rectangle):
            # print(_, "포함됨")
            # 조건문
            if p[0] < int((rectangle[3][0] - rectangle[0][0]) / 2):
                result_txt = '왼쪽'
                print(f'{result_txt}에 {result_type}있습니다.')
                result_case.append([result_txt, result_type])
                break
            elif int((rectangle[3][0] - rectangle[0][0]) / 2) < p[0]:
                result_txt = '오른쪽'
                result_case.append([result_txt, result_type])
                print(f'{result_txt}에 {result_type}있습니다.')
                break
        else:
            pass
            # print("포함되지 않음")

result_case


True
오른쪽에 사람있습니다.
True
오른쪽에 사람있습니다.
True
오른쪽에 사람있습니다.
False
False
False
False


[['오른쪽', '사람'], ['오른쪽', '사람'], ['오른쪽', '사람']]

In [20]:
for _ in range(0, len(result_case)):
    a = result_case[_][0]
    ob = result_case[_][1]

이미지 테스트

In [23]:
import os
import cv2
import numpy as np
import sys

filePath  = show_result

img = cv2.imread(filePath)

w, h, c = img.shape

mask = np.zeros((w,h,c), dtype=np.uint8)

# pts1 = np.array([[270, 310], [12, 445], [610, 445], [383, 310]])
# pts1 = np.array([[12, 310], [270, 445], [383, 445], [610, 310]])
pts1 = np.array([[280, 290], [20, 470], [620, 470], [380, 290]])



desired_alpha = 90 
alpha = desired_alpha / 255.0 

# 다각형 그리기
polyline = cv2.polylines(mask, [pts1], isClosed=True, color=(0, 0, 255), thickness=5)

# 마스크 채우기
mask1 = cv2.fillPoly(polyline, [pts1], (255,0,0))
img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2BGRA)
result = cv2.addWeighted(img, 1, mask1, alpha, 0)

# 이미지 화면 출력
cv2.imshow('img', result)

# 이미지 저장
cv2.imwrite('./result.jpg', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [107]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

img_path = './1_17_7-00020.jpg'
results = model.predict(img_path)

# yolo식 클래스id
yolo_class = model.names

# box 정보 리스트화 <- 여기서 나오는 데이터는 텐서배열
ls_point = []
ls_id = []
for _ in results:
    # box 정보 얻기
    for a, b in zip(_.boxes.xywh.tolist(), _.boxes.cls.tolist()):
        ls_point.append(a)
        ls_id.append(b)

# 불필요한 인덱스 파악
drop_index = []
index = 0
for _ in ls_id:
    id = int(_)
    if id == 0 or id == 1 or id == 2\
        or id == 3 or id == 5 or id == 7:
        pass
    else:
        drop_index.append(index)
    index += 1
# 불필요한 클래스가 있으면 삭제 해야하므로 인덱스는 뒤집기
drop_index.reverse()

if len(drop_index) > 0:
# 불필요한 인덱스 삭제
    for _ in drop_index:
        ls_point.pop(_)
        ls_id.pop(_)

# 필요한 정보 <- 비율 좌표가 아닌 절대좌표
re_id = []
point = []
for one, two in zip(ls_point, ls_id):

    # class_id
    id = int(two)
    # 변경하는 이유는 gtts에서 한글로 변환해야되서.
    if id == 0:
        id = '사람'
    elif id == 1:
        id = '자전거'
    elif id == 2:
        id = '자동차'
    elif id == 3:
        id = '오토바이'
    elif id == 5:
        id = '버스'
    elif id == 7:
        id = '트럭'     
    re_id.append(id)

    # yolo식 절대좌표
    cen_x = int(one[0])
    cen_y = int(one[1])
    w = int(one[2])
    h = int(one[3])

    # 사각형의 4개의 점 좌표 구하기.
    # x1 : 왼쪽 상단, x2 : 오른쪽 상단
    # x3 : 왼쪽 하단, x4 : 오른쪽 하단
    x1 = (int(cen_x - (w/2)), int(cen_y + (h/2)))
    x2 = (int(cen_x + (w/2)), int(cen_y + (h/2)))
    x3 = (int(cen_x - (w/2)), int(cen_y - (h/2)))
    x4 = (int(cen_x + (w/2)), int(cen_y - (h/2)))
    
    point.append((x1, x2, x3, x4))

Ultralytics YOLOv8.0.20  Python-3.10.13 torch-2.1.1+cpu CPU


YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


In [64]:
model = YOLO('./train14_best.pt')

video_path = './1_28_22.mp4'
results = model.predict(video_path, save=True)

Ultralytics YOLOv8.0.20  Python-3.10.13 torch-2.1.1+cpu CPU
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs
Results saved to runs\detect\predict5


In [165]:
# !pip install gTTS
# !pip install playsound

In [4]:
if 'rectangle' in locals():
    print('있습니다.')
else:
    print('없습니다.')

없습니다.


In [6]:
import winsound as wsd

wsd.Beep(2000, 3000)
print('0------------')
print('end')

def play_beep():
    wsd.Beep(2000, 500)

0------------
end


### test 이미지를 읽고 알고리즘 만드는건 구현 완료되어있는데 cv2에서 frame을 model.predcit(frame) 해서 해결할수있다는 것을 확인.. 했지만 cv2 에서 frame은 넘파이 형식으로 나오는데... 그래도 가능하나? test ㄱ

In [1]:
import cv2, os
from ultralytics import YOLO
from shapely.geometry import Polygon, Point
from gtts import gTTS
import playsound
import numpy as np
import winsound as wsd
import threading


# 한글을 영어로 변환함수
def en_to_ko(result_case):

    # 방향 한글 -> 영어
    way = result_case[0][0]
    if way == '오른쪽':
        en_way = 'right'
    elif way == '왼쪽':
        en_way = 'left'
    
    # 객체 한글 -> 영어
    id = result_case[0][1]
    if id == '사람':
        en_id = 'person'
    elif id == '자전거':
        en_id = 'bicycle'
    elif id == '자동차':
        en_id = 'car'
    elif id == '오토바이':
        en_id = 'motorcycle'
    elif id == '자전거':
        en_id = 'person'
    elif id == '버스':
        en_id = 'bus'
    elif id == '트럭':
        en_id = 'truck'
    
    return en_way, en_id

# 비프음
def play_beep():
    wsd.Beep(2000, 500)
    # print('beep 쓰레드 종료')
# tts
def play_sound(path):
    try:
        playsound.playsound(path)
        # print('tts 쓰레드 종료')
    except:
        pass
# 워닝존 in 탐지
def warning_zone(point, polygon):
    """
    point: [x, y] 좌표 값을 가진 리스트
    polygon: 다각형의 꼭지점 좌표를 가진 리스트
    """

    # 점의 좌표를 사용하여 점의 객체를 만듭니다.
    point_obj = Point(point)

    # 다각형의 꼭지점 좌표를 사용하여 다각형의 경계를 구합니다.
    polygon_path = Polygon(polygon)

    # 점의 객체가 다각형의 경계 안에 있는지 확인합니다.
    result = polygon_path.contains(point_obj)
    # print(result)
    # print('워닝존 : ', polygon)
    return result

model = YOLO('./train14_best.pt')

video_path = './1_28_22.mp4'
# video_path = './1_20_1.mp4'
# video_path = './test.avi'
video = cv2.VideoCapture(video_path)

if not video:
    raise(f'동영상 못 읽지 못했습니다.\n경로는 {video_path} 입니다.')

# 비디오 정보
video_w = video.get(cv2.CAP_PROP_FRAME_WIDTH)
video_h = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

# 왼쪽 위 [0], 왼쪽 밑 [1], 오른쪽 위 [3], 오른쪽 밑 [2]
# rectangle = [(180, 480), (20, 570), (620, 570), (480, 480)]
# rectangle = [(180, 480), (70, 570), (570, 570), (480, 480)]
roatio_yolo = [[0.28125, 0.75], [0.109375, 0.890625], [0.890625, 0.890625], [0.75, 0.75]]

# 워닝존 비율 좌표로 변경 <- 640 x 640 일때 워닝존 좌표이므로
# 변수가 있을때
if 'rectangle' in locals():    
    ratio_point = []
    for _ in rectangle:
        x, y = _[0], _[1]
        convert_x = x / video_w
        convert_y = y / video_h
        ratio_point.append([convert_x, convert_y])
else:
    pass

# warning_zone 함수에는 해당 비율 좌표가 아닌 절대 좌표가 필요하므로 convert 작업 필요.
if 'ratio_point' in locals():
    abs_point = []
    for _ in ratio_point:
        x, y = _[0], _[1]

        convert_x = int(x * video_w)
        convert_y = int(y * video_h)
        abs_point.append([convert_x, convert_y])
else:
    pass

if 'roatio_yolo' in locals():
    abs_point = []
    for _ in roatio_yolo:
        x, y = _[0], _[1]

        convert_x = int(x * video_w)
        convert_y = int(y * video_h)
        abs_point.append([convert_x, convert_y])
else:
    pass

t_gtts = None
t_beep = None

while True:
    # 동영상 정보 가져옴
    ret, frame = video.read()

    # 동영상 다음 프레임이 없다는것이므로 종료
    if not ret:
        break
    
    results = model.predict(frame)

    # yolo식 클래스id
    yolo_class = model.names

    # box 정보 리스트화 <- 여기서 나오는 데이터는 텐서배열
    ls_point = []
    ls_id = []
    for _ in results:
        # box 정보 얻기
        for a, b in zip(_.boxes.xywh.tolist(), _.boxes.cls.tolist()):
            ls_point.append(a)
            ls_id.append(b)

    # 불필요한 인덱스 파악
    drop_index = []
    index = 0
    for _ in ls_id:
        id = int(_)
        if id == 0 or id == 1 or id == 2\
            or id == 3 or id == 5 or id == 7:
            pass
        else:
            drop_index.append(index)
        index += 1
    # 불필요한 클래스가 있으면 삭제 해야하므로 인덱스는 뒤집기
    drop_index.reverse()

    if len(drop_index) > 0:
    # 불필요한 인덱스 삭제
        for _ in drop_index:
            ls_point.pop(_)
            ls_id.pop(_)

    # 필요한 정보 <- 비율 좌표가 아닌 절대좌표
    re_id = []
    rectangle_point = []
    for one, two in zip(ls_point, ls_id):

        # class_id
        id = int(two)
        # 변경하는 이유는 gtts에서 한글로 변환해야되서.
        if id == 0:
            id = '사람'
        elif id == 1:
            id = '자전거'
        elif id == 2:
            id = '자동차'
        elif id == 3:
            id = '오토바이'
        elif id == 5:
            id = '버스'
        elif id == 7:
            id = '트럭'     
        re_id.append(id)

        # yolo식 절대좌표
        cen_x = int(one[0])
        cen_y = int(one[1])
        w = int(one[2])
        h = int(one[3])

        # 사각형의 4개의 점 좌표 구하기.
        # x1 : 왼쪽 상단, x2 : 오른쪽 상단
        # x3 : 왼쪽 하단, x4 : 오른쪽 하단
        x1 = (int(cen_x - (w/2)), int(cen_y + (h/2)))
        x2 = (int(cen_x + (w/2)), int(cen_y + (h/2)))
        x3 = (int(cen_x - (w/2)), int(cen_y - (h/2)))
        x4 = (int(cen_x + (w/2)), int(cen_y - (h/2)))
        
        rectangle_point.append((x1, x2, x3, x4))

    result_case = []
    
    # 객체가 들어왔을때
    object_in = False
    # 좌표가 사다리꼴 안에 있는지 확인합니다.
    detection_left = 0
    detection_right = 0
    for point, result_type in zip(rectangle_point, re_id):
        x1 = point[0]
        x4 = point[3]

        for p in point:
            point_obj = Point(p)
            # yolo 박스 전체 시각화 (빨강색)
            cv2.rectangle(frame, x1, x4, color=(0,0,255), thickness=2)
            if warning_zone(point_obj, abs_point):
                object_in = True
                object_x = p[0]
                zone_x = int((abs_point[2][0] + abs_point[1][0]) / 2)
                # 조건문
                if object_x < zone_x:
                    # 워닝존 들어온 박스만 시각화(녹색)
                    cv2.rectangle(frame, x1, x4, color=(0,255,0), thickness=2)
                    detection_left += 1
                    result_txt = '왼쪽'
                    # print(f'{result_txt}에 {result_type}')
                    result_case.append([result_txt, result_type])
                    break
                elif object_x > zone_x:
                    cv2.rectangle(frame, x1, x4, color=(0,255,0), thickness=2)
                    detection_right += 1
                    result_txt = '오른쪽'
                    # print(f'{result_txt}에 {result_type}')
                    result_case.append([result_txt, result_type])
                    break
                # print(result_case)
            else:  
                object_in = False
    # print(detection_left, detection_right, end='\r')
    # center
    if detection_left > 0 and detection_right > 0:
        txt_point = (int(video_w * (2/5)), int(video_h * (1/3)))
        cv2.putText(frame, 'both Warning', org=txt_point, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)

    elif detection_left > 0 and detection_right == 0:
        txt_point = (int(video_w * (1/10)), int(video_h * (1/3)))
        cv2.putText(frame, 'left Warning', org=txt_point, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)

    elif detection_left == 0 and detection_right > 0:
        txt_point = (int(video_w * (2/3)), int(video_h * (1/3)))
        cv2.putText(frame, 'right Warning', org=txt_point, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)
    
    detection_left = 0
    detection_right = 0

    zone = [np.array(abs_point)]
    cv2.polylines(frame, zone, True ,(255,255,255), thickness=3)
    cv2.imshow("YOLOv8", frame)

    key = cv2.waitKey(30)

    if key == 27:
        break
    elif key == 0x260000:
        speed += 1
    elif key == 0x280000:
        speed -= 0.5
        if speed < 0.1:
            speed = 0.1

    # 동영상에서 워닝존안에 객체가 안들어오면 result_case의 정보는 없다 
    if len(result_case) == 0:
        pass
    else:
        txt = result_case[0][0] + ' ' + result_case[0][1]
        way, id = en_to_ko(result_case)
        path = f'./blackbox_sound/{way}_{id}.mp3'

        # 해당 파일이 없으면 비프음
        if os.path.exists(path) == False:
            # 파이썬 내장 사운드 삐이익 재생
            if t_beep !=None and t_beep.is_alive():
                continue
            else:
                t_beep = threading.Thread(target=play_beep)
                t_beep.start()
            # tts = gTTS(txt, lang='ko')
            # tts.save(path)
        else:
            if t_gtts !=None and t_gtts.is_alive():
                continue
            else:
                t_gtts = threading.Thread(target=play_sound, args=(path, ))
                t_gtts.start()

cv2.destroyAllWindows()

Ultralytics YOLOv8.0.20  Python-3.10.13 torch-2.1.1+cpu CPU
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs

    Error 259 for command:
        play ./blackbox_sound/right_person.mp3 wait
    지정한 명령 매개 변수를 드라이버가 인식할 수 없습니다.

    Error 259 for command:
        play ./blackbox_sound/right_person.mp3 wait
    지정한 명령 매개 변수를 드라이버가 인식할 수 없습니다.

    Error 259 for command:
        play ./blackbox_sound/right_person.mp3 wait
    지정한 명령 매개 변수를 드라이버가 인식할 수 없습니다.

    Error 259 for command:
        play ./blackbox_sound/right_person.mp3 wait
    지정한 명령 매개 변수를 드라이버가 인식할 수 없습니다.

    Error 259 for command:
        play ./blackbox_sound/right_person.mp3 wait
    지정한 명령 매개 변수를 드라이버가 인식할 수 없습니다.

    Error 259 for command:
        play ./blackbox_sound/right_person.mp3 wait
    지정한 명령 매개 변수를 드라이버가 인식할 수 없습니다.

    Error 259 for command:
        play ./blackbox_sound/right_person.mp3 wait
    지정한 명령 매개 변수를 드라이버가 인식할 수 없습니다.



    Error 263 for command:
        close ./blackbox_sound/right_person.mp3
    지정한 장치가 열려 있지 않거나 MCI에서 인식되지 않습니다.
Failed to close the file: ./blackbox_sound/right_person.mp3


# 버전 - 1
워닝존에 객체 존재시 알고리즘 구현(워닝존 안에 객체가 있는지 -> 워닝존 밑변 기준 x 값 왼쪽에 있는지, 오른쪽에 있는지)

In [ ]:
from shapely.geometry import Polygon, Point

def warning_zone(point, polygon):
    """
    point: [x, y] 좌표 값을 가진 리스트
    polygon: 다각형의 꼭지점 좌표를 가진 리스트
    """

    # 점의 좌표를 사용하여 점의 객체를 만듭니다.
    point_obj = Point(point)

    # 다각형의 꼭지점 좌표를 사용하여 다각형의 경계를 구합니다.
    polygon_path = Polygon(polygon)

    # 점의 객체가 다각형의 경계 안에 있는지 확인합니다.
    result = polygon_path.contains(point_obj)
    print(result)
    # print('워닝존 : ', polygon)
    return result

# 워닝존
rectangle = [(280, 290), (20, 470), (620, 470), (380, 290)]

# 데이터 클래스 하나 4개의 점
# square = point

result_case = []
# 좌표가 사다리꼴 안에 있는지 확인합니다.
for square, result_type in zip(point, object_type):
    for p in square:
        point_obj = Point(p)
        if warning_zone(point_obj, rectangle):
            # print(_, "포함됨")
            # 조건문
            if p[0] < int((rectangle[3][0] - rectangle[0][0]) / 2):
                result_txt = '왼쪽'
                print(f'{result_txt}에 {result_type}있습니다.')
                result_case.append([result_txt, result_type])
                break
            elif int((rectangle[3][0] - rectangle[0][0]) / 2) < p[0]:
                result_txt = '오른쪽'
                result_case.append([result_txt, result_type])
                print(f'{result_txt}에 {result_type}있습니다.')
                break
        else:
            pass
            # print("포함되지 않음")

result_case

# 버전 2 - 공식문서 변경으로 정보 추출 파트 수정 

In [19]:
import cv2, os
from ultralytics import YOLO
from shapely.geometry import Polygon, Point
from gtts import gTTS
import playsound
import numpy as np
import winsound as wsd
import threading


# 한글을 영어로 변환함수
def en_to_ko(result_case):

    # 방향 한글 -> 영어
    way = result_case[0][0]
    if way == '오른쪽':
        en_way = 'right'
    elif way == '왼쪽':
        en_way = 'left'
    
    # 객체 한글 -> 영어
    id = result_case[0][1]
    if id == '사람':
        en_id = 'person'
    elif id == '자전거':
        en_id = 'bicycle'
    elif id == '자동차':
        en_id = 'car'
    elif id == '오토바이':
        en_id = 'motorcycle'
    elif id == '자전거':
        en_id = 'person'
    elif id == '버스':
        en_id = 'bus'
    elif id == '트럭':
        en_id = 'truck'
    
    return en_way, en_id

# 비프음
def play_beep():
    wsd.Beep(2000, 500)
    # print('beep 쓰레드 종료')
# tts
def play_sound(path):
    try:
        playsound.playsound(path)
        # print('tts 쓰레드 종료')
    except:
        pass
# 워닝존 in 탐지
def warning_zone(point, polygon):
    """
    point: [x, y] 좌표 값을 가진 리스트
    polygon: 다각형의 꼭지점 좌표를 가진 리스트
    """

    # 점의 좌표를 사용하여 점의 객체를 만듭니다.
    point_obj = Point(point)

    # 다각형의 꼭지점 좌표를 사용하여 다각형의 경계를 구합니다.
    polygon_path = Polygon(polygon)

    # 점의 객체가 다각형의 경계 안에 있는지 확인합니다.
    result = polygon_path.contains(point_obj)
    # print(result)
    # print('워닝존 : ', polygon)
    return result

t_gtts = None
t_beep = None
def warning(model, video_path):

    global t_gtts
    global t_beep

    model = YOLO(model)

    video = cv2.VideoCapture(video_path)

    if not video:
        raise(f'동영상 못 읽지 못했습니다.\n경로는 {video_path} 입니다.')

    # 비디오 정보
    video_w = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    video_h = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

    # 왼쪽 위 [0], 왼쪽 밑 [1], 오른쪽 위 [3], 오른쪽 밑 [2]
    # rectangle = [(180, 480), (20, 570), (620, 570), (480, 480)]
    # rectangle = [(180, 480), (70, 570), (570, 570), (480, 480)]

    roatio_yolo = [[0.28125, 0.75], [0.109375, 0.890625], [0.890625, 0.890625], [0.75, 0.75]]

    # 워닝존 비율 좌표로 변경 <- 640 x 640 일때 워닝존 좌표이므로
    # 변수가 있을때
    if 'rectangle' in locals():    
        ratio_point = []
        for _ in rectangle:
            x, y = _[0], _[1]
            convert_x = x / video_w
            convert_y = y / video_h
            ratio_point.append([convert_x, convert_y])
    else:
        pass

    # warning_zone 함수에는 해당 비율 좌표가 아닌 절대 좌표가 필요하므로 convert 작업 필요.
    if 'ratio_point' in locals():
        abs_point = []
        for _ in ratio_point:
            x, y = _[0], _[1]

            convert_x = int(x * video_w)
            convert_y = int(y * video_h)
            abs_point.append([convert_x, convert_y])
    else:
        pass

    if 'roatio_yolo' in locals():
        abs_point = []
        for _ in roatio_yolo:
            x, y = _[0], _[1]

            convert_x = int(x * video_w)
            convert_y = int(y * video_h)
            abs_point.append([convert_x, convert_y])
    else:
        pass



    while True:
        # 동영상 정보 가져옴
        ret, frame = video.read()

        # 동영상 다음 프레임이 없다는것이므로 종료
        if not ret:
            break
        
        results = model.predict(frame)

        # yolo식 클래스id
        yolo_class = model.names

        # box 정보 리스트화 <- 여기서 나오는 데이터는 텐서배열
        ls_point = []
        ls_id = []
        for _ in results:
            # box 정보 얻기
            for a, b in zip(_.boxes.xywh.tolist(), _.boxes.cls.tolist()):
                ls_point.append(a)
                ls_id.append(b)

        # 불필요한 인덱스 파악
        drop_index = []
        index = 0
        for _ in ls_id:
            id = int(_)
            if id == 0 or id == 1 or id == 2\
                or id == 3 or id == 5 or id == 7:
                pass
            else:
                drop_index.append(index)
            index += 1
        # 불필요한 클래스가 있으면 삭제 해야하므로 인덱스는 뒤집기
        drop_index.reverse()

        if len(drop_index) > 0:
        # 불필요한 인덱스 삭제
            for _ in drop_index:
                ls_point.pop(_)
                ls_id.pop(_)

        # 필요한 정보 <- 비율 좌표가 아닌 절대좌표
        re_id = []
        rectangle_point = []
        for one, two in zip(ls_point, ls_id):

            # class_id
            id = int(two)
            # 변경하는 이유는 gtts에서 한글로 변환해야되서.
            if id == 0:
                id = '사람'
            elif id == 1:
                id = '자전거'
            elif id == 2:
                id = '자동차'
            elif id == 3:
                id = '오토바이'
            elif id == 5:
                id = '버스'
            elif id == 7:
                id = '트럭'     
            re_id.append(id)

            # yolo식 절대좌표
            cen_x = int(one[0])
            cen_y = int(one[1])
            w = int(one[2])
            h = int(one[3])

            # 사각형의 4개의 점 좌표 구하기.
            # x1 : 왼쪽 상단, x2 : 오른쪽 상단
            # x3 : 왼쪽 하단, x4 : 오른쪽 하단
            x1 = (int(cen_x - (w/2)), int(cen_y + (h/2)))
            x2 = (int(cen_x + (w/2)), int(cen_y + (h/2)))
            x3 = (int(cen_x - (w/2)), int(cen_y - (h/2)))
            x4 = (int(cen_x + (w/2)), int(cen_y - (h/2)))
            
            rectangle_point.append((x1, x2, x3, x4))

        result_case = []

        # 좌표가 사다리꼴 안에 있는지 확인합니다.
        detection_left = 0
        detection_right = 0
        for point, result_type in zip(rectangle_point, re_id):
            x1 = point[0]
            x4 = point[3]

            for p in point:
                point_obj = Point(p)
                # yolo 박스 전체 시각화 (빨강색)
                cv2.rectangle(frame, x1, x4, color=(0,0,255), thickness=2)
                if warning_zone(point_obj, abs_point):
                    object_in = True
                    object_x = p[0]
                    zone_x = int((abs_point[2][0] + abs_point[1][0]) / 2)
                    # 조건문
                    if object_x < zone_x:
                        # 워닝존 들어온 박스만 시각화(녹색)
                        cv2.rectangle(frame, x1, x4, color=(0,255,0), thickness=2)
                        detection_left += 1
                        result_txt = '왼쪽'
                        # print(f'{result_txt}에 {result_type}')
                        result_case.append([result_txt, result_type])
                        break
                    elif object_x > zone_x:
                        cv2.rectangle(frame, x1, x4, color=(0,255,0), thickness=2)
                        detection_right += 1
                        result_txt = '오른쪽'
                        # print(f'{result_txt}에 {result_type}')
                        result_case.append([result_txt, result_type])
                        break
                else:  
                    pass

        # center
        if detection_left > 0 and detection_right > 0:
            txt_point = (int(video_w * (2/5)), int(video_h * (1/3)))
            cv2.putText(frame, 'both Warning', org=txt_point, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)

        elif detection_left > 0 and detection_right == 0:
            txt_point = (int(video_w * (1/10)), int(video_h * (1/3)))
            cv2.putText(frame, 'left Warning', org=txt_point, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)

        elif detection_left == 0 and detection_right > 0:
            txt_point = (int(video_w * (2/3)), int(video_h * (1/3)))
            cv2.putText(frame, 'right Warning', org=txt_point, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)
        
        detection_left = 0
        detection_right = 0

        zone = [np.array(abs_point)]
        cv2.polylines(frame, zone, True ,(255,255,255), thickness=3)
        cv2.imshow("YOLOv8", frame)

        key = cv2.waitKey(60)

        if key == 27:
            break

        # 동영상에서 워닝존안에 객체가 안들어오면 result_case의 정보는 없다 
        if len(result_case) == 0:
            pass
        else:
            txt = result_case[0][0] + ' ' + result_case[0][1]
            way, id = en_to_ko(result_case)
            path = f'./blackbox_sound/{way}_{id}.mp3'

            # 해당 파일이 없으면 비프음
            if os.path.exists(path) == False:
                # 파이썬 내장 사운드 삐이익 재생
                if t_beep !=None and t_beep.is_alive():
                    continue
                else:
                    t_beep = threading.Thread(target=play_beep)
                    t_beep.start()
                # tts = gTTS(txt, lang='ko')
                # tts.save(path)
            else:
                if t_gtts !=None and t_gtts.is_alive():
                    continue
                else:
                    t_gtts = threading.Thread(target=play_sound, args=(path, ))
                    t_gtts.start()

    cv2.destroyAllWindows()

In [ ]:
model = './train14_best.pt'
video_path = './1_20_1.mp4'
video_path= './test.avi'
warning(model, video_path)